In [1]:
import torch 
import numpy as np 
import pandas as pd

from src.scm.distribution import *
from src.scm.scm import *
from src.causal_graph import *
from src.scm.feedforward_ncm import *
from src.queries import *

In [2]:
X,Y,Z,W = 'X','Y','Z','W'
Ur,Uz,Ux,Uy = 'Ur', 'Uz', 'Ux', 'Uy'

V = [Z,X,Y]
U = [Ur,Uz,Ux,Uy]

class ExDist(Distribution):
    def __init__(self, u):
        super().__init__(u)
        self.rand_state = np.random.RandomState()
        self.p = {
            Ur: 0.25,
            Uz: 0.95,
            Ux: 0.9,
            Uy: 0.7
        }

    def sample(self, n=1, device=None):
        return {u: T.from_numpy(self.rand_state.binomial(1, self.p[u], size=(n))).long() for u in self.u}

pu = ExDist(U)

def fz(v, u): 
    return torch.tensor([int(u[Ur][i]==1 and u[Uz][i]==1) for i in range(len(u[Ur]))])
def fx(v, u): 
    return torch.tensor([int(v[Z][i]==u[Ux][i]) for i in range(len(v[Z]))])
def fy(v, u): 
    # print(v[X])
    # cond1 = [int(v[X][i]==1 and u[Ur][i]==1) for i in range(len(u[Ur]))]
    def cond1(i): return v[X][i]==1 and u[Ur][i]==1
    def cond2(i): return v[X][i]==0 and u[Ur][i]==1 and u[Uy][i]==1
    def cond3(i): return v[X][i]==0 and u[Ur][i]==0 and u[Uy][i]==0
    return torch.tensor([int(cond1(i) or cond2(i) or cond3(i)) for i in range(len(u[Ur]))])

f = {
    X: fx,
    Y: fy,
    Z: fz
}

scm = SCM(V,f,pu)
uval=[0,0,0,0]
u={U[i]:torch.tensor([uval[i]]) for i in range(len(U))}
# u=pu.sample(n=3)
# scm(u=u,do={X:torch.tensor([1])})

In [3]:
pz = 0.007125 + 0.016625 + 0.064125 + 0.149625
px = 0.001125 + 0.002625 + 0.021375 + 0.049875 + 0.000375 + 0.000875 + 0.064125 + 0.149625

print(pz)
print(px)

0.23750000000000002
0.29000000000000004


In [4]:
U = scm.pu.sample(n=10000)
def compute_ctf(ncm, var, do={}, conditions={}, n=10000, get_prob=True, u=None):
    U, n_new = get_conditioned_u(ncm, conditions=conditions, n=n, u=u)

    samples = dict()
    expanded_do_terms = dict()
    for k in do:
        if k == "nested":
            # TODO
            expanded_do_terms.update(compute_ctf("based on ctf term in v", get_prob=False))
        else:
            expanded_do_terms[k] = expand_do(do[k],n_new)
        # instead of update based on select=var, update select=terms of this do, in case of nested
    samples = ncm(n=None, u=U, do=expanded_dos(do,n_new))

    if get_prob is False: return samples
    return tensor_prob_dist(samples[var])

conditions={}
_, pycondx1 = compute_ctf(scm, Y, conditions={X:1}, u=U)
_, pyx1 = compute_ctf(scm, Y, do={X:1}, conditions=conditions, u=U)
_, pyx0 = compute_ctf(scm, Y, do={X:0}, conditions=conditions, u=U)
print(pycondx1)
print(pyx1)
print(pyx0)
pyx1 - pyx0

tensor([0.2486, 0.7514])
tensor([0.7446, 0.2554])
tensor([0.5958, 0.4042])


tensor([ 0.1488, -0.1488])

In [5]:
def sample_ctf(ncm, term: CTFTerm, conditions={}, n=10000, u=None, get_prob=True):
    U, n = get_conditioned_u(ncm, u, conditions=conditions, n=n)

    expanded_do_terms = dict()
    for k in term.do_vals:
        if k == "nested":
            nested_term = term.do_vals[k]
            ctf = sample_ctf(ncm=ncm, term=nested_term, n=n, u=U, get_prob=False)
            for var in nested_term.vars:
                expanded_do_terms.update({var: ctf[var]})
        else:
            expanded_do_terms[k] = expand_do(term.do_vals[k],n)

    return ncm(n=None, u=U, do=expanded_do_terms, select=term.vars, evaluating=True)

syx1 = sample_ctf(scm, CTFTerm([Y], {X:1}), conditions=conditions, u=U)
# summ = sum(syx1[Y].tolist())
# length = len(syx1[Y].tolist())
# print(f'{summ} / {length} = {summ/length}')

_, px1 = tensor_prob_dist(syx1[Y])

sycondx1 = sample_ctf(scm, CTFTerm([Y]), conditions={X:1}, u=U)
_, pycondx1 =tensor_prob_dist(sycondx1[Y])

syx0 = sample_ctf(scm, CTFTerm([Y], {X:0}), conditions=conditions, u=U)
_, px0 = tensor_prob_dist(syx0[Y])
print(pycondx1)
print(px1)
print(px0)

tensor([0.2486, 0.7514])
tensor([0.7446, 0.2554])
tensor([0.5958, 0.4042])


In [6]:
_, py_condx1 = prob_ctf(Y, scm, CTFTerm([Y]), conditions={X:1}, u=U)
_, py_dox1 = prob_ctf(Y, scm, CTFTerm([Y], {X:1}), u=U)
print(py_condx1, py_dox1)

tensor([0.2486, 0.7514]) tensor([0.7446, 0.2554])
